In [1]:
from pyecharts import options as opts
from pyecharts.charts import Bar, Grid
from pyecharts import options as opts
from pyecharts.globals import ThemeType
from pyecharts.commons.utils import JsCode
import pandas as pd

# CSV文件路径
csv_file = "快乐再出发弹幕.csv"

# 使用Pandas读取CSV文件，指定编码为utf-8-sig
df = pd.read_csv(csv_file, encoding='utf-8-sig')
df['每期播放量'] = df['每期播放量'].str.replace('万', '').astype(float)
df['每期时长'] = df['每期时长'].apply(lambda x: sum(int(x) * 60 ** i for i, x in enumerate(reversed(x.split(':')))))
# 打印DataFrame，包含从CSV文件中读取的数据
print(df[:5])


    节目名称  是否需要VIP    节目ID                                              每期背景图  \
0  快乐再出发        1  457910  https://2img.hitv.com/preview/sp_images/2022/0...   
1  快乐再出发        1  457910  https://2img.hitv.com/preview/sp_images/2022/0...   
2  快乐再出发        1  457910  https://2img.hitv.com/preview/sp_images/2022/0...   
3  快乐再出发        1  457910  https://2img.hitv.com/preview/sp_images/2022/0...   
4  快乐再出发        1  457910  https://2img.hitv.com/preview/sp_images/2022/0...   

    每期播放量            每期标题        发布日期                          每期详细标题  每期时长  \
0  3685.5  先导片：0713兄弟集合出发  2022-07-05  快乐再出发先导片：0713兄弟集合出发！张远王栎鑫糗事大曝光  1444   
1  3685.5  先导片：0713兄弟集合出发  2022-07-05  快乐再出发先导片：0713兄弟集合出发！张远王栎鑫糗事大曝光  1444   
2  3685.5  先导片：0713兄弟集合出发  2022-07-05  快乐再出发先导片：0713兄弟集合出发！张远王栎鑫糗事大曝光  1444   
3  3685.5  先导片：0713兄弟集合出发  2022-07-05  快乐再出发先导片：0713兄弟集合出发！张远王栎鑫糗事大曝光  1444   
4  3685.5  先导片：0713兄弟集合出发  2022-07-05  快乐再出发先导片：0713兄弟集合出发！张远王栎鑫糗事大曝光  1444   

        期ID                                 

#### 1、每期时常与播放量

In [2]:
playcnt = df[['每期标题', '每期时长', '每期播放量']].drop_duplicates().sort_values('每期播放量', ascending=False)
names = list(playcnt['每期标题'])
times = list(playcnt['每期时长'])
counts = list(playcnt['每期播放量'])
ar = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
    .add_xaxis(names)
    .add_yaxis("时长", times)
    .add_yaxis("播放量", counts)
    .set_series_opts(label_opts=opts.LabelOpts(position="top"))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="每期时长与播放量对比"),
        xaxis_opts=opts.AxisOpts(name="标题",
                                axislabel_opts=opts.LabelOpts(rotate=45, interval=0)),
        yaxis_opts=opts.AxisOpts(name="时长（秒），播放量（万）"),
        toolbox_opts=opts.ToolboxOpts(),
    )
)
grid=Grid()
# 仅使用pos_top修改相对顶部的位置
grid.add(ar,grid_opts=opts.GridOpts(pos_left="10%", pos_bottom="25%"))
grid.render("1.每期时长与播放量对比.html")

'D:\\PythonProject\\柳州水位通告\\芒果tv弹幕分析\\1.每期时长与播放量对比.html'

#### 2、整体弹幕数量分布

In [4]:
danmu_counts = df.groupby("每期标题")["弹幕内容"].count().reset_index()

# 重命名列名，以更清晰表示数据
danmu_counts.columns = ["每期标题", "弹幕数量"]
danmu_counts = danmu_counts.sort_values('弹幕数量', ascending=False)
names = list(danmu_counts['每期标题'])
counts = list(danmu_counts['弹幕数量'])
ar = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT, height="700px", width="1000px"))
    .add_xaxis(names)
    .add_yaxis("弹幕数量", counts)
    .reversal_axis()
    .set_series_opts(label_opts=opts.LabelOpts(position="right"))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="每期时常与播放量对比"),
        xaxis_opts=opts.AxisOpts(name="标题"),
        yaxis_opts=opts.AxisOpts(name="弹幕数量",
                                axislabel_opts=opts.LabelOpts(rotate=-0, interval=0)),
        toolbox_opts=opts.ToolboxOpts(),
    )
)
grid=Grid()
# 仅使用pos_top修改相对顶部的位置
grid.add(ar,grid_opts=opts.GridOpts(pos_left="20%"))
grid.render("2.整体弹幕数量分布.html")

'D:\\PythonProject\\柳州水位通告\\芒果tv弹幕分析\\2.整体弹幕数量分布.html'

#### 3、弹幕量Top10

In [5]:
danmu_counts = df["弹幕内容"].value_counts().reset_index()
# 重命名列名，以更清晰表示数据
danmu_counts.columns = ["弹幕内容", "数量"]
# 获取前十位弹幕内容和对应的数量
top_10_danmu = danmu_counts.head(10)
names = list(top_10_danmu['弹幕内容'])
counts = list(top_10_danmu['数量'])
ar = (
    Bar()
    .add_xaxis(names)
    .add_yaxis("数量", counts)
    .reversal_axis()
    .set_series_opts(label_opts=opts.LabelOpts(position="right"))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="弹幕量Top10"),
        xaxis_opts=opts.AxisOpts(name="弹幕内容"),
        yaxis_opts=opts.AxisOpts(name="数量",
                                axislabel_opts=opts.LabelOpts(rotate=20, interval=0)),
    toolbox_opts=opts.ToolboxOpts(),)
)
grid=Grid()
# 仅使用pos_top修改相对顶部的位置
grid.add(ar,grid_opts=opts.GridOpts(pos_left="30%"))
grid.render("3.弹幕量Top10.html")

'D:\\PythonProject\\柳州水位通告\\芒果tv弹幕分析\\3.弹幕量Top10.html'

#### 4、弹幕点赞量Top10

In [6]:
# 按照弹幕点赞量降序排序，并获取前十个弹幕
top_10_danmu = df[["弹幕内容", "弹幕点赞量"]].sort_values(by="弹幕点赞量", ascending=False).head(10)
names = list(top_10_danmu['弹幕内容'])
counts = list(top_10_danmu['弹幕点赞量'])
ar = (
    Bar()
    .add_xaxis(names)
    .add_yaxis("弹幕点赞量", counts)
    .reversal_axis()
    .set_series_opts(label_opts=opts.LabelOpts(position="right"))
    .set_global_opts(
        title_opts=opts.TitleOpts(title="弹幕点赞量Top10"),
        xaxis_opts=opts.AxisOpts(name="弹幕内容"),
        yaxis_opts=opts.AxisOpts(name="弹幕点赞量",
                                axislabel_opts=opts.LabelOpts(rotate=0, interval=0)),
    toolbox_opts=opts.ToolboxOpts(),)
)
grid=Grid()
# 仅使用pos_top修改相对顶部的位置
grid.add(ar,grid_opts=opts.GridOpts(pos_left="20%"))
grid.render("4.弹幕点赞量Top10.html")

'D:\\PythonProject\\柳州水位通告\\芒果tv弹幕分析\\4.弹幕点赞量Top10.html'

In [87]:
result = df.groupby('每期标题').agg({'弹幕内容':'count', '所属分钟段':pd.Series.mode})

In [89]:
result.sort_values(by='弹幕内容', ascending=False)

,弹幕内容,所属分钟段
每期标题,,
第2期：喜剧人版渔民号子上线,62752,29_30
第3期：影视城打工记,62058,67_68
第1期：0713兄弟荒岛之旅,60616,"[1_2, 78_79]"
第4期：兄弟团海边夜聊,39417,"[0_1, 10_11, 11_12]"
第7期：兄弟团海边演唱会,31293,"[37_38, 38_39, 43_44, 47_48, 68_69, 72_73]"
第5期：俞灏明姚政空降,30992,"[49_50, 68_69]"
第6期：“王伴奏”遭哄抢？,30927,45_46
加更版：《活该》首发,20410,"[18_19, 23_24]"
先导片：0713兄弟集合出发,16796,"[0_1, 12_13]"
